<a href="https://colab.research.google.com/github/mdavistr-cmu/Davis-Troller_DSPN_S25/blob/main/ExerciseSubmissions/Davis-Troller_Exercise12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/CoAxLab/Data-Explorations/blob/main/book/exercises/cross-validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise 12: Cross validation
-----

In this exercise, we'll practice implementing cross validation techniques, including leave-one-out and k-fold cross validation. We'll use the `PimaIndiansDiabetes2` practice dataset, which has medical data on a group of Pima Native American women, including whether or not they have diabetes. This dataset is part of the `mlbench` package. We'll be using each person's medical history to predict whether or not they have been diagnosed with diabetes.

# 1: Data (1 pts)
---

Load the `tidyverse`, `boot`, and `mlbench` packages (you may need to install `boot` and `mlbench`).

Load the `PimaIndiansDiabetes2` dataset using the `data()` function. Drop the `insulin` column (it just has a lot of missing data) and then drop `NA`s from the rest of the dataset. Save your updated dataset to a new variable name. Finally, print the dimensions of your new dataset, and look at the first few lines of data.

In [1]:
library(tidyverse)
install.packages("boot")
install.packages("mlbench")
library(boot)
library(mlbench)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
data("PimaIndiansDiabetes2")
df <- PimaIndiansDiabetes2 %>%
  select(-insulin) %>%
  drop_na()

head(df)
dim(df)


,pregnant,glucose,pressure,triceps,mass,pedigree,age,diabetes
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,6,148,72,35,33.6,0.627,50,pos
2,1,85,66,29,26.6,0.351,31,neg
4,1,89,66,23,28.1,0.167,21,neg
5,0,137,40,35,43.1,2.288,33,pos
7,3,78,50,32,31.0,0.248,26,pos
9,2,197,70,45,30.5,0.158,53,pos


[1] 532   8

(Note that in medical contexts, `pedigree` refers to a system of measuring family history of a condition. So here, higher numbers mean greater family history of diabetes. You can read more about this dataset [here](https://rdrr.io/cran/mlbench/man/PimaIndiansDiabetes.html).)

# 2. Leave-one-out Cross Validation (4 pts)

In the tutorial, we learned how to fit leave-one-out cross validation using the `cv.glm` function from the `boot` package. But we can also do this manually using `predict()` like we have in the past.

Let's predict `diabetes`, a dichotomous outcome, using all the other variables in our modified dataset.

First, fit a logistic regression model using all of the observations except the very first one. Then use your fitted model to predict whether your holdout case is positive or negative for diabetes. Remember that logistic regression coefficients are in **log-odds**, meaning that if an output is positive, the probability of the outcome is greater than 50%; if the output is negative, the probability of the outcome is less than 50%.

Compare your result to the actual response in row one above. Did your model correctly classify this observation?

In [4]:
model <- glm(diabetes ~ ., data = df[-1, ], family = binomial)

log_odds_pred1 <- predict(model, newdata = df[1, ], type = "link")
print(log_odds_pred1)


predicted_class <- ifelse(log_odds_pred1 > 0, "pos", "neg")
print(paste("Predicted class:", predicted_class))


actual_class <- df$diabetes[1]
print(paste("Actual class:", actual_class))


correct <- predicted_class == actual_class
print(paste("Correct classification?", correct))



        1 
0.9920213 
[1] "Predicted class: pos"
[1] "Actual class: pos"
[1] "Correct classification? TRUE"


So we just calculated a single iteration of LOOCV. We used 531 rows of our data to fit a model to predict the outcome of the last row.

Below, use a `for` loop to iterate through the rest of your dataset doing the same thing. You will need to:
* Create a data frame `results` with two columns: one named `actual` which holds the true classification for each observation, and one named `predicted`, which should be filled with `NA`s. This is where you'll store the output of your loop.
* Create a loop that runs through each row of your data, pulls that observation out, trains your model on the remaining data, and then tests the fitted model on your test observation.
* Store your model *predictions* ("pos" or "neg" -- not the log-odds) in the `predicted` column of your `results` dataframe

After you run your loop, print the first few lines of `results`.

In [5]:
# Initialize `results` data frame
results <- data.frame(actual = df$diabetes, predicted = NA)

#for loop
for (i in 1:nrow(df)){
    # separate individual observation `i` from the rest of your data
    trainData <- df[-i, ]
    testData <- df[i, , drop = FALSE]
    # train your model
     loocv_model <- glm(diabetes ~ ., data = trainData, family = binomial)

    # test model on hold out observation
     log_odds_pred <- predict(loocv_model, newdata = testData, type = "link")

    # classify model prediction as "pos" or "neg" and add to `results`
    predicted_class <- ifelse(log_odds_pred > 0, "pos", "neg")
    results$predicted[i] <- predicted_class

}

head(results)




,actual,predicted
,<fct>,<chr>
1,pos,pos
2,neg,neg
3,neg,neg
4,pos,pos
5,pos,neg
6,pos,pos


Now, calculate the overall error of your model. What proportion of cases were incorrectly classified?

In [6]:
errorRate <- mean(results$actual != results$predicted)

print(errorRate)

[1] 0.2218045


# 3. Compare to `cv.glm` (3 pts)

Now, let's compare this result to the `cv.glm` function. Using the tutorial as a guide, use `cv.glm` to run LOOCV on the data, using the same model (i.e., still using all of the variables to predict diabetes diagnosis).

Note that, because this is a `classification` problem and not a regression problem like in the tutorial, we need to adjust the `cost` argument of `cv.glm`. We can read more about this in the docs:

In [12]:
#?cv.glm
glm.fit = glm(diabetes ~ ., data = df, family = binomial)

#cost_function <- function(actual, predicted) {
 # mean((actual == "pos" & predicted < 0.5) | (actual == "neg" & predicted >= 0.5))
#} caused me to have a 00.00 error rate

cost_function <- function(actual, predicted) {
  actual_binary <- ifelse(actual == "pos", 1, 0)  # Convert to 0/1
  predicted_binary <- ifelse(predicted >= 0.5, 1, 0)  # Convert to 0/1
  mean(actual_binary != predicted_binary)  # Misclassification rate
}


# Perform LOOCV with cv.glm
loocv_results <- cv.glm(df, glm.fit, cost = cost_function)

# Extract LOOCV error estimate
cv_error <- loocv_results$delta[1]

# Print the cross-validation error
print(paste("LOOCV Error Rate (cv.glm):", round(cv_error, 4)))

# Print manual error rate from previous result
manual_error_rate <- mean(results$actual != results$predicted)
print(paste("Manual LOOCV Error Rate:", round(manual_error_rate, 4)))
print(paste("cv.glm LOOCV Error Rate:", round(cv_error, 4)))





[1] "LOOCV Error Rate (cv.glm): 0.2613"
[1] "Manual LOOCV Error Rate: 0.2218"
[1] "cv.glm LOOCV Error Rate: 0.2613"


Here, we see `cost` is defined as:
> "A function of two vector arguments specifying the cost function for the cross-validation. The first argument to cost should correspond to the **observed responses** and the second argument should correspond to the **predicted or fitted responses** from the generalized linear model."

In the example code (scroll to bottom of the docs), we see that the appropriate cost function for a binary classification is

``
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
``

Where `r` is the vector of observed responses (technically "pos" and "neg", but R treats these as 1 and 0 under the hood), and `pi` is the vector of *probabilities* (not log-odds) fit by the model. Thus, this boils down to our error: what proportion of observations were incorrectly classified. You will need to include this code below.

In [14]:
cost <- function(r, pi = 0) mean(abs(r-pi) > 0.5)
loocv_results <- cv.glm(df, glm.fit, cost = cost)
print(paste("LOOCV Error Rate (cv.glm):", round(cv_error, 4)))

[1] "LOOCV Error Rate (cv.glm): 0.2613"


How do your results compare to your manual LOOCV above?

> The results of the cv.glm function had a higher error rate by 4% than the manual LOOCV.


# 4. Adjusting K and Reflection (2 pts)

Recall that LOOCV has some drawbacks. In particular, it has quite high *variance* which can lead to poor performance on new test data. We can reduce this variance by increasing K.

Below, re-run your cross validation using `cv.glm` with `k` set to 3, 5, 10, and 15.

In [18]:
set.seed(1)
#INSERT CODE BELOW

# K = 3
loocv_results3 <- cv.glm(df, glm.fit, cost = cost_function, K=3)
cv_error3 <- loocv_results3$delta[1]
print(paste("LOOCV Error Rate (cv.glm) w k = 3:", round(cv_error3, 4)))
# K = 5
loocv_results5 <- cv.glm(df, glm.fit, cost = cost_function, K=5)
cv_error5 <- loocv_results5$delta[1]
print(paste("LOOCV Error Rate (cv.glm) w k = 5:", round(cv_error5, 4)))
# K = 10
loocv_results10 <- cv.glm(df, glm.fit, cost = cost_function, K=10)
cv_error10 <- loocv_results10$delta[1]
print(paste("LOOCV Error Rate (cv.glm) w k = 10:", round(cv_error10, 4)))

# K = 15
loocv_results15 <- cv.glm(df, glm.fit, cost = cost_function, K=15)
cv_error15 <- loocv_results15$delta[1]
print(paste("LOOCV Error Rate (cv.glm) w k = 15:", round(cv_error15, 4)))

[1] "LOOCV Error Rate (cv.glm) w k = 3: 0.2688"
[1] "LOOCV Error Rate (cv.glm) w k = 5: 0.2594"
[1] "LOOCV Error Rate (cv.glm) w k = 10: 0.2594"
[1] "LOOCV Error Rate (cv.glm) w k = 15: 0.2632"


#### Reflection

How do your errors compare to your LOOCV error above? How do they change as k increases?
> They decrease and then go back up.

If you change the random seed above, you'll get slightly different errors. If you were to do the same with your LOOCV above , would you expect to get different results each time? Why or why not?
> LOOCV will always get the same result because it's deterministic (n-1).
> k-Fold CV (e.g., k=3, 5, 10, 15) will have different results depending on random seed due to the random way the fold splits.


**DUE:** 5pm March 25, 2024

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here.
> *Someone's Name*
>
>
